# Bag of Words

### Introduction

In this lesson, we'll get started with our first NLP dataset, explaining how we can transform it so that we can use it in machine learning.  We'll do so by using the newsgroups dataset.  The dataset consists of Internet message board posts, which we'll try to predict the category of.  To determine the category of each post, we'll use the bag of words technique.  With bag of words, we represent each message board posts with a histogram that represents the frequency of each word in the post.  Let's get started.

### Loading our Data

We'll begin by loading up some data from the newsgroups dataset.

In [10]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

documents = pd.DataFrame(newsgroups_train['data'], columns = ['text'])
y = newsgroups_train['target']

Let's look at the first observation.

In [15]:
first_document = documents['text'][0]
first_document

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

We can see that it consists of text of a message board post.  It's contents include the sender's email address, the subject, and the content of the post.

In an NLP problem, an observation like this is referred to as a **document**.

> In NLP, a **document** is a distinct text. This generally means that each article, book, or so on is its own document.  It is the equivalent of an observation in machine learning.

### Segmenting our Document

Now to extract features from each document, it seems like we would break it into it's components.  When we break our document into chunks, whether sentences, or lines, or heading, body and footer, it's called **segmentation**.  To begin, we'll focus on breaking our text into tokens. 

>  A **token** is a string of contiguous characters between two spaces, or between a space and punctuation marks.

So, `will not` can be broken into the tokens of `will` and `not`, but `won't` can also be broken into two tokens, `won`, `'t`.  To start, we'll simply segment each document into tokens by splitting the text with pure Python.

> These are some of the tokens from the first document.

In [16]:
tokenized_doc = str.split(first_document)
tokenized_doc[6:10]

['WHAT', 'car', 'is', 'this!?']

### Encoding Each Word

Now, as we know, we can't simply pass this text into a machine learning classifier, like logistic regression.  Rather we have to represent each token as a different number.  For example, `What` may be represented by the number `0`, and `car` may be represented by the number `3`, etc.  And we could represent those words with a list where each the 0 index represents the number of occurrences of `What`, and the third index is dedicated to `car`:

In [ ]:
[1, 0, 0, 1]
# What    car

> The way that we translate our words into text is called **text representation**.  And when we represent our documents as a vector, its called a **vector space model**.

To automate this process for us, we can use the `CountVectorizer` from sklearn.

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(newsgroups_train.data)

> We can see that the count vectorizer has the same API as our other sklearn transformers.  We initialize the transformer and call `fit_transform` to transform our data.

If we look at our data now, we'll see that each document is now represented by a vector of numbers.

In [18]:
X[0:2].toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

So those first two vectors represent our first two documents.  How did it do this?  Well if we look at the dimensions of our vectors, we'll see that each document is represented by a vector of length 130107, one entry for each word in the entire set of documents, called the **corpus**.  

> The entire set of documents is called the **corpus**.

In reading all of the documents, the count vectorizer formed a `vocabulary`, where each word is represented by a number. 

In [88]:
list(vectorizer.vocabulary_.items())[:10]

[('from', 56979),
 ('lerxst', 75358),
 ('wam', 123162),
 ('umd', 118280),
 ('edu', 50527),
 ('where', 124031),
 ('my', 85354),
 ('thing', 114688),
 ('subject', 111322),
 ('what', 123984)]

So above, we can see that the word `from` is represented by `56979`.  Let's see how many times it appeared in the first document.

In [89]:
X[0].toarray()[0][56979]

3

Apparently it appeared three times.

> The bag-of-words model is a simplifying representation used in natural language processing and information retrieval (IR). In this model, a text (such as a sentence or a document) is represented as the bag (multiset) of its words, disregarding grammar and even word order but keeping multiplicity. 

> The bag-of-words model is commonly used in methods of document classification where the (frequency of) occurrence of each word is used as a feature for training a classifier.

### Does it really work?

So we just saw a pretty simple technique for taking a document and translating it into a word vector.  We just assign every token in the corpus to a number -- collected into our dictionary -- and use a vector to represent the count of the words in the document.

Note that in making this translation some information is lost.  For example, here is the original version of the third document.

In [23]:
# vectors[3].
documents.iloc[3, 0]

'From: jgreen@amber (Joe Green)\nSubject: Re: Weitek P9000 ?\nOrganization: Harris Computer Systems Division\nLines: 14\nDistribution: world\nNNTP-Posting-Host: amber.ssd.csd.harris.com\nX-Newsreader: TIN [version 1.1 PL9]\n\nRobert J.C. Kyanko (rob@rjck.UUCP) wrote:\n> abraxis@iastate.edu writes in article <abraxis.734340159@class1.iastate.edu>:\n> > Anyone know about the Weitek P9000 graphics chip?\n> As far as the low-level stuff goes, it looks pretty nice.  It\'s got this\n> quadrilateral fill command that requires just the four points.\n\nDo you have Weitek\'s address/phone number?  I\'d like to get some information\nabout this chip.\n\n--\nJoe Green\t\t\t\tHarris Corporation\njgreen@csd.harris.com\t\t\tComputer Systems Division\n"The only thing that really scares me is a person with no sense of humor."\n\t\t\t\t\t\t-- Jonathan Winters\n'

And this is the inverse transformation of it.

In [114]:
vectorizer.inverse_transform(vectors[3])

[array(['14', '734340159', 'about', 'abraxis', 'address', 'amber',
        'anyone', 'article', 'as', 'chip', 'class1', 'com', 'command',
        'computer', 'corporation', 'csd', 'distribution', 'division', 'do',
        'edu', 'far', 'fill', 'four', 'from', 'get', 'goes', 'got',
        'graphics', 'green', 'harris', 'have', 'host', 'humor', 'iastate',
        'in', 'information', 'is', 'it', 'jgreen', 'joe', 'jonathan',
        'just', 'know', 'kyanko', 'level', 'like', 'lines', 'looks', 'low',
        'me', 'newsreader', 'nice', 'nntp', 'no', 'number', 'of', 'only',
        'organization', 'p9000', 'person', 'phone', 'pl9', 'points',
        'posting', 'pretty', 'quadrilateral', 're', 'really', 'requires',
        'rjck', 'rob', 'robert', 'scares', 'sense', 'some', 'ssd', 'stuff',
        'subject', 'systems', 'that', 'the', 'thing', 'this', 'tin', 'to',
        'uucp', 'version', 'weitek', 'winters', 'with', 'world', 'writes',
        'wrote', 'you'], dtype='<U180')]

So we can see that the order of text is lost.  But if we are just looking at this unordered blob of text, perhaps we can get a sense of what. this is about.  With words like `systems`, `distribution`, `graphics`, and `p9000`, it seems like this might be something related to computers.

And we could imagine that if each of these words is represented as a feature, with simple one hot encoding, as CountVectorizer applies, then perhaps a logistic regression model could classify our text based on the amount of each token.

Let's give it a shot.  

### Training a model

We'll begin by splitting our data into a training and testing set.

In [92]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify = y)

Then we can load up a logistic regression model, and change the number of iterations.

In [93]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

Finally, we can score the model.

In [95]:
lr.score(X_test, y_test)

0.8709677419354839

### Summary

In this lesson we learned about the bag of words model for representing text.  We did so by being introduced to the NLP problem in general.  We saw that we describe each observation as a **document**, and break our text into different **tokens**, here by splitting on spaces and punctuation marks.  We then translated the tokens in the document into a text representation of bag of words.

With bag of words,  we represent each document through a vector, where each index represents a different word.  To do so, we must first have each token in the corpus be represented by number.  And for a document, we specify the count of words at the correct index.  

We encoded our documents using the `CountVectorizer` from the `sklearn.feature_extraction.text` module.  We fed the CountVectorizer our corpus of words, and it returned to us the corresponding document vectors.  Then we fed these document vectors into our logistic regression model to classify each document.

### Resources

[Naive Bayes Newsgroups](https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a)

[NLP Newsgroups](https://medium.com/@siyao_sui/nlp-with-the-20-newsgroups-dataset-ab35cd0ea902)



[FastAI NLP](https://github.com/fastai/course-nlp/blob/master/2-svd-nmf-topic-modeling.ipynb)

[Sklearn text data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

[Naive Bayes Classifier](https://towardsdatascience.com/the-naive-bayes-classifier-e92ea9f47523)